# Реализовать алгоритм распознающий направление стрелочек из датасета ha.zip

In [29]:
import os
import cv2
import numpy as np

def calculate_iou(boxA, boxB):
    """Calculate Intersection over Union (IoU) between two bounding boxes."""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    if (boxAArea + boxBArea - interArea) == 0:
        return 0

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def preprocess_image(image):
    """Convert an image to binary (black and white)."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def detect_arrow_shape(thresh):
    """Check if the image contains an arrow shape."""
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 500:  # Minimum threshold to filter out noise
            continue

        # Approximate the contour into a polygon
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)

        # If the polygon has between 5 to 9 edges, it could be an arrow
        if 5 <= len(approx) <= 9:
            return True

    return False  

def get_reference_boxes():
    """Create a list of reference bounding boxes from sample images."""
    directions = ["LEFT", "RIGHT", "UP", "DOWN"]
    ref_boxes = {}

    for direction in directions:
        img = cv2.imread(f"arrow_{direction.lower()}.png")  # Read sample image
        
        if img is None:
            continue
        
        thresh = preprocess_image(img)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            cnt = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(cnt)
            ref_boxes[direction] = (x, y, x + w, y + h)
            print(f"Reference Box {direction}: {ref_boxes[direction]}")

    return ref_boxes

def classify_arrow(arrow_image, ref_boxes):
    """Classify the arrow direction (left, right, up, down)."""
    thresh = preprocess_image(arrow_image)

    # If no arrow shape is detected, return "Unknown"
    if not detect_arrow_shape(thresh):
        return "Unknown"

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return "Unknown"  # No object found

    cnt = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(cnt)
    arrow_box = (x, y, x + w, y + h)

    print(f"Arrow Box: {arrow_box}")

    candidate_directions = ["LEFT", "RIGHT", "UP", "DOWN"]
    best_direction = "Unknown"
    best_iou = 0

    for direction in candidate_directions:
        if direction in ref_boxes:
            iou = calculate_iou(arrow_box, ref_boxes[direction])
            print(f"IoU with {direction}: {iou:.4f}")

            if iou > best_iou:
                best_iou = iou
                best_direction = direction

    return best_direction  

# Load reference bounding boxes from sample images
ref_boxes = get_reference_boxes()

# Read input image
arrow_img = cv2.imread("unzipped_images/left/67.png")  
direction = classify_arrow(arrow_img, ref_boxes)
print("Arrow direction:", direction)


Reference Box LEFT: (27, 49, 168, 171)
Reference Box RIGHT: (32, 73, 244, 254)
Reference Box UP: (52, 17, 137, 179)
Reference Box DOWN: (23, 22, 877, 878)
Arrow Box: (26, 43, 162, 163)
IoU with LEFT: 0.8488
IoU with RIGHT: 0.2721
IoU with UP: 0.5128
IoU with DOWN: 0.0223
Arrow direction: LEFT


Этот код предназначен для распознавания направления стрелки на изображении. Основные этапы:

1. Предварительная обработка изображения

    * Преобразование в градации серого.
    * Бинаризация с помощью Otsu’s Thresholding.
2. Определение наличия стрелки

    * Поиск контуров и их аппроксимация.
    * Фильтрация по количеству углов (5–9).
3. Сравнение с эталонными изображениями

    * Определение ограничивающих рамок (bounding boxes) эталонных стрелок.
    * Использование IoU (Intersection over Union) для определения наиболее похожего направления.

* Текущая система работает хорошо в простых случаях, но испытывает трудности с распознаванием стрелок, указывающих направо, из-за следующих ограничений:

* Ограничения, влияющие на распознавание стрелки вправо
 1. IoU недостаточно точен
Анализируется только ограничивающий прямоугольник, без детального рассмотрения формы.
 2. Не учитываются ключевые особенности формы
Не выделяются головка стрелки, симметрия и точное направление.
Ошибки при наклоне или незначительном изменении формы.
 3. Не выполняется нормализация угла наклона
Отсутствует этап выравнивания изображения перед распознаванием.
Смещение ограничивающего прямоугольника приводит к неверному определению направления.